In [3]:
import numpy as np
import pandas as pd 
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from alpaca_trade_api.rest import TimeFrame 
import pandas_ta as pta
import DateTime
from datetime import date, timedelta
from sklearn.preprocessing import StandardScaler
from pandas.tseries.offsets import DateOffset
from sklearn.metrics import classification_report

In [4]:
# Creating aplaca api key variables for access 
load_dotenv()
api_key = os.getenv('ALPACA_API_KEY')
api_secret = os.getenv('ALPACA_SECRET_KEY')

#creating timeframe to recieve most recent data
yesterday_date = date.today() - timedelta(1)


2022-09-30


In [24]:
#Setting necessary variables for alpaca api function
base_url = 'https://paper-api.alpaca.markets'
api = tradeapi.REST(api_key, api_secret, base_url, api_version='v2')
account = api.get_account()





# Create the Alpaca API object
ETH_price = api.get_crypto_bars('ETHUSD',  TimeFrame.Day, "2014-06-08", yesterday_date)




,exchange,open,high,low,close,volume,trade_count,vwap
timestamp,,,,,,,,
2022-09-29 05:00:00+00:00,ERSX,1342.53,1349.63,1289.58,1326.18,1346.364223,309,1327.355385
2022-09-29 05:00:00+00:00,FTXU,1341.80,1352.90,1287.20,1328.50,85850.336000,38650,1325.077029
2022-09-30 05:00:00+00:00,CBSE,1328.54,1373.46,1315.00,1329.00,379867.670201,425669,1339.895294
2022-09-30 05:00:00+00:00,ERSX,1331.06,1372.56,1315.07,1329.88,1818.861066,532,1345.633777
2022-09-30 05:00:00+00:00,FTXU,1328.50,1373.60,1309.80,1328.90,61372.641000,42117,1339.683789


In [73]:
# Creating Function to clean data
def get_dataframe(raw_data):
    raw_data = raw_data.df
    raw_data = raw_data.drop(columns=["open","high","low","volume","trade_count","vwap"])
    raw_data = raw_data[raw_data['exchange'].str.contains("ERSX|FTXU")==False]
    raw_data["Actual Returns"] = raw_data["close"].pct_change()
    raw_data["RSI EMA"] = pta.rsi(raw_data["close"], length = 30)
    raw_data["RSI Drift"] = pta.rsi(raw_data["close"], length = 30, drift = 10)
    raw_data = raw_data.sort_values("timestamp", axis=0, ascending=False)
    raw_data = raw_data.dropna()
    return raw_data

In [74]:
get_dataframe(ETH_price)

,exchange,close,Actual Returns,RSI EMA,RSI Drift
timestamp,,,,,
2022-09-30 05:00:00+00:00,CBSE,1329.00,0.000271,44.166330,29.674832
2022-09-29 05:00:00+00:00,CBSE,1328.64,-0.009483,44.153952,29.724385
2022-09-28 05:00:00+00:00,CBSE,1341.36,0.044787,44.490839,29.870477
2022-09-27 05:00:00+00:00,CBSE,1283.86,-0.070058,42.576306,29.453016
2022-09-26 05:00:00+00:00,CBSE,1380.58,0.066711,45.105937,30.288123
...,...,...,...,...,...
2016-07-01 05:00:00+00:00,CBSE,12.14,-0.014610,45.277279,44.116832
2016-06-30 05:00:00+00:00,CBSE,12.32,-0.029921,45.679074,45.800332
2016-06-29 05:00:00+00:00,CBSE,12.70,0.038430,46.521565,45.339459
